In [ ]:
%pip install cvlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 54.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for cvlib: filename=cvlib-0.2.7-py3-none-any.whl size=10046367 sha256=e739113f269661308bfa6f8d1e848189c723d7de202f66f3a5c1d6d4534f3b2f
  Stored in directory: /root/.cache/pip/wheels/9e/a5/d4/fe37b48fe4f4b621ba5e574a991230070f3cc4f02322a01489
  Created wheel for progressbar: filename=progressbar-2.5-py3-none-any.whl size=12067 sha256=5b6d01012e565b54b4bb42f36c709b8169f5a8ecd90d8c349243217630acb71c
  Stored in directory: /root/.cache/pip/wheels/cd/17/e5/765d1a3112ff3978f70223502f6047e06c43a24d7c5f8ff95b
Successfully built cvlib progressbar


In [ ]:
from tensorflow.keras.utils import img_to_array
from keras.models import load_model
from keras.utils import get_file
import numpy as np
import cv2
import os
import cvlib as cv

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dwnld_link = "https://github.com/arunponnusamy/cvlib/releases/download/v0.2.0/gender_detection.model"
model_path = get_file("gender_detection.model", dwnld_link,
                     cache_subdir="pre-trained", cache_dir=os.getcwd())



104188200/104188200 [==============================] - 6s 0us/step


In [ ]:
images_folder = "./drive/MyDrive/Screenshots/Inception"

In [ ]:
images = []
for image_filename in os.listdir(images_folder):
  image = cv2.imread(os.path.join(images_folder,image_filename))
  if image is None:
    print(image_filename)
    pass
  images.append(image)
print(images)



Streaming output truncated to the last 5000 lines.

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8), array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [2, 0, 0],
        [2, 0, 0],
        [2, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [2, 0, 0],
        [2, 0, 0],
        [2, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [2, 0, 0],
        [2, 0, 0],
        [2, 0, 0]]], dtype=uint8), arra

In [ ]:
len(images)

1336

In [ ]:
images[0]

array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8)

In [ ]:
# load pre-trained model
model = load_model(model_path)

In [ ]:
output_dir = "./drive/MyDrive/Screenshots/output"
inception_output_dir = os.path.join(output_dir, "Inception")
os.mkdir(output_dir)
os.mkdir(inception_output_dir)

In [ ]:
os.mkdir(os.path.join(inception_output_dir, "men"))
os.mkdir(os.path.join(inception_output_dir, "women"))
os.mkdir(os.path.join(inception_output_dir, "mixed"))

In [ ]:
classes = ['man','woman']
counter = 0

In [ ]:
import copy

In [ ]:
for image in copy.deepcopy(images):
  # detect faces in the image
  try:
    face, confidence = cv.detect_face(image)
  except TypeError:
    pass

  if len(face) == 0:
    pass

  genders = []

  # loop through detected faces
  for idx, f in enumerate(face):

    try:
      # get corner points of face rectangle
      (startX, startY) = f[0], f[1]
      (endX, endY) = f[2], f[3]

      # draw rectangle over face
      cv2.rectangle(image, (startX,startY), (endX,endY), (0,255,0), 2)

      # crop the detected face region
      face_crop = np.copy(image[startY:endY,startX:endX])

      # preprocessing for gender detection model
      face_crop = cv2.resize(face_crop, (96,96))
      face_crop = face_crop.astype("float") / 255.0
      face_crop = img_to_array(face_crop)
      face_crop = np.expand_dims(face_crop, axis=0)

      # apply gender detection on face
      conf = model.predict(face_crop)[0]
      # print(conf)
      # print(classes)

      # get label with max accuracy
      idx = np.argmax(conf)
      label = classes[idx]

      genders.append(classes[idx])

      label = "{}: {:.2f}%".format(label, conf[idx] * 100)

      Y = startY - 10 if startY - 10 > 10 else startY + 10

      # write label and confidence above face rectangle
      cv2.putText(image, label, (startX, Y),  cv2.FONT_HERSHEY_SIMPLEX,
                  0.7, (0, 255, 0), 2)
    except Exception:
      pass

  print(genders)
  pass

  if all([gender == "man" for gender in genders]):
    cv2.imwrite(os.path.join(output_dir, "Inception", "men", str(counter) +".jpg"), image)
  elif all([gender == "woman" for gender in genders]):
    cv2.imwrite(os.path.join(output_dir, "Inception", "woman", str(counter) + ".jpg"), image)
  else:
    cv2.imwrite(os.path.join(output_dir, "Inception", "mixed", str(counter) + ".png"), image)
  counter += 1
  print("saved image")


1/1 [==============================] - 0s 20ms/step
['man']
saved image
1/1 [==============================] - 0s 20ms/step
['man']
saved image
1/1 [==============================] - 0s 24ms/step
['man']
saved image
1/1 [==============================] - 0s 28ms/step
['man']
saved image
1/1 [==============================] - 0s 19ms/step
['man']
saved image
1/1 [==============================] - 0s 20ms/step
['man']
saved image
1/1 [==============================] - 0s 24ms/step
['man']
saved image
1/1 [==============================] - 0s 26ms/step
['man']
saved image
1/1 [==============================] - 0s 21ms/step
['man']
saved image
1/1 [==============================] - 0s 22ms/step
['man']
saved image
1/1 [==============================] - 0s 20ms/step
['man']
saved image
1/1 [==============================] - 0s 19ms/step
['man']
saved image
1/1 [==============================] - 0s 21ms/step
['man']
saved image
1/1 [==============================] - 0s 21ms/step
['man']
save

error: ignored

In [ ]:
# display output
cv2.imshow("gender detection", image)

# press any key to close window
cv2.waitKey()

# save output
cv2.imwrite("gender_detection.jpg", image)

# release resources
cv2.destroyAllWindows()